# Desenvolvimento do modelo de predição de preço

Exploração inicial de modelos de base

In [21]:
import pandas as pd
from dagshub.data_engine import datasources
import mlflow
import dagshub
from sklearn.model_selection import train_test_split
import mlflow.sklearn
import mlflow.catboost
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import mlflow.models.signature
from mlflow.models import infer_signature
from catboost import CatBoostRegressor

## Carregando Dataset

In [25]:
ds = datasources.get_datasource("michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl", "processed")

INFO:gql.transport.requests:>>> {"query": "query datasource($id: ID, $name: String) {\n  datasource(id: $id, name: $name) {\n    id\n    name\n    rootUrl\n    integrationStatus\n    preprocessingStatus\n    metadataFields {\n      name\n      valueType\n      multiple\n      tags\n    }\n    type\n  }\n}", "variables": {"id": null, "name": "processed"}}
INFO:gql.transport.requests:<<< {"data":{"datasource":[{"id":5799,"name":"processed","rootUrl":"repo://michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl/main:data/processed","integrationStatus":"VALID","preprocessingStatus":"READY","metadataFields":[{"name":"size","valueType":"INTEGER","multiple":false,"tags":null},{"name":"media type","valueType":"STRING","multiple":false,"tags":null}],"type":"REPOSITORY"}]}}


In [23]:
ds.all().dataframe

INFO:gql.transport.requests:>>> {"query": "query datasource($id: ID, $name: String) {\n  datasource(id: $id, name: $name) {\n    id\n    name\n    rootUrl\n    integrationStatus\n    preprocessingStatus\n    metadataFields {\n      name\n      valueType\n      multiple\n      tags\n    }\n    type\n  }\n}", "variables": {"id": 5799, "name": "processed"}}
INFO:gql.transport.requests:<<< {"data":{"datasource":[{"id":5799,"name":"processed","rootUrl":"repo://michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl/main:data/processed","integrationStatus":"VALID","preprocessingStatus":"READY","metadataFields":[{"name":"size","valueType":"INTEGER","multiple":false,"tags":null},{"name":"media type","valueType":"STRING","multiple":false,"tags":null}],"type":"REPOSITORY"}]}}


Output()

INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/repos/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl "HTTP/1.1 200 OK"
INFO:gql.transport.requests:>>> {"query": "query datasourceQuery($datasource: ID!, $queryInput: QueryInput, $first: Int, $after: String) {\n  datasourceQuery(\n    datasource: $datasource\n    filter: $queryInput\n    first: $first\n    after: $after\n  ) {\n    edges {\n      node {\n        id\n        path\n        metadata {\n          key\n          value\n          timeZone\n        }\n      }\n    }\n    pageInfo {\n      hasNextPage\n      endCursor\n    }\n    queryDataTime\n    selectFields {\n      name\n      originalName\n      autoGenerated\n      valueType\n      asOf\n      multiple\n      tags\n    }\n  }\n}", "variables": {"datasource": 5799, "queryInput": {"query": null}, "first": 5000, "after": null}}


INFO:httpx:HTTP Request: POST https://dagshub.com/api/internal/trackAnalyticsEvent "HTTP/1.1 200 OK"


INFO:gql.transport.requests:<<< {"data":{"datasourceQuery":{"edges":[{"node":{"id":102330336,"path":"laptop-price-brl-processed.csv","metadata":[{"key":"media type","value":"text/plain","timeZone":""},{"key":"size","value":51778,"timeZone":""}]}}],"pageInfo":{"hasNextPage":false,"endCursor":null},"queryDataTime":1751074929,"selectFields":[{"name":"media type","originalName":"media type","autoGenerated":true,"valueType":"STRING","asOf":null,"multiple":false,"tags":null},{"name":"size","originalName":"size","autoGenerated":true,"valueType":"INTEGER","asOf":null,"multiple":false,"tags":null}]}}}


,path,datapoint_id,dagshub_download_url,media type,size
0,laptop-price-brl-processed.csv,102330336,https://dagshub.com/api/v1/repos/michelpf/fiap...,text/plain,51778


In [24]:
res = ds.head()

for dp in res:
    dataset_url = dp.download_url

INFO:gql.transport.requests:>>> {"query": "query datasource($id: ID, $name: String) {\n  datasource(id: $id, name: $name) {\n    id\n    name\n    rootUrl\n    integrationStatus\n    preprocessingStatus\n    metadataFields {\n      name\n      valueType\n      multiple\n      tags\n    }\n    type\n  }\n}", "variables": {"id": 5799, "name": "processed"}}
INFO:gql.transport.requests:<<< {"data":{"datasource":[{"id":5799,"name":"processed","rootUrl":"repo://michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl/main:data/processed","integrationStatus":"VALID","preprocessingStatus":"READY","metadataFields":[{"name":"size","valueType":"INTEGER","multiple":false,"tags":null},{"name":"media type","valueType":"STRING","multiple":false,"tags":null}],"type":"REPOSITORY"}]}}


Output()

INFO:gql.transport.requests:>>> {"query": "query datasourceQuery($datasource: ID!, $queryInput: QueryInput, $first: Int, $after: String) {\n  datasourceQuery(\n    datasource: $datasource\n    filter: $queryInput\n    first: $first\n    after: $after\n  ) {\n    edges {\n      node {\n        id\n        path\n        metadata {\n          key\n          value\n          timeZone\n        }\n      }\n    }\n    pageInfo {\n      hasNextPage\n      endCursor\n    }\n    queryDataTime\n    selectFields {\n      name\n      originalName\n      autoGenerated\n      valueType\n      asOf\n      multiple\n      tags\n    }\n  }\n}", "variables": {"datasource": 5799, "queryInput": {"query": null}, "first": 100, "after": null}}


INFO:httpx:HTTP Request: POST https://dagshub.com/api/internal/trackAnalyticsEvent "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dagshub.com/api/internal/trackAnalyticsEvent "HTTP/1.1 200 OK"


INFO:gql.transport.requests:<<< {"data":{"datasourceQuery":{"edges":[{"node":{"id":102330336,"path":"laptop-price-brl-processed.csv","metadata":[{"key":"media type","value":"text/plain","timeZone":""},{"key":"size","value":51778,"timeZone":""}]}}],"pageInfo":{"hasNextPage":false,"endCursor":null},"queryDataTime":1751074932,"selectFields":[{"name":"media type","originalName":"media type","autoGenerated":true,"valueType":"STRING","asOf":null,"multiple":false,"tags":null},{"name":"size","originalName":"size","autoGenerated":true,"valueType":"INTEGER","asOf":null,"multiple":false,"tags":null}]}}}


In [26]:
dataset_url

'https://dagshub.com/api/v1/repos/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl/raw/main/data/processed/laptop-price-brl-processed.csv'

In [27]:
df = pd.read_csv(dataset_url)
df.head()

,ram_gb,ssd,hdd,graphic_card_gb,warranty,price,brand_asus,brand_dell,brand_hp,brand_lenovo,...,ram_type_other,os_other,os_windows,os_bit_32-bit,os_bit_64-bit,weight_casual,weight_gaming,weight_thinnlight,touchscreen_0,touchscreen_1
0,4,0,1024,0,0,2321,1,0,0,0,...,0,0,1,0,1,1,0,0,1,0
1,4,0,1024,0,0,2613,0,0,0,1,...,0,0,1,0,1,1,0,0,1,0
2,4,0,1024,0,0,2680,0,0,0,1,...,0,0,1,0,1,1,0,0,1,0
3,8,512,0,2,0,4689,1,0,0,0,...,0,0,1,1,0,1,0,0,1,0
4,4,0,512,0,0,1808,1,0,0,0,...,0,0,1,0,1,1,0,0,1,0


## Desenvolvimento e experimentos de modelos

In [28]:
dagshub.init(repo_owner="michelpf", repo_name="fiap-ds-mlops-10dtsr-laptop-pricing-brl", mlflow=True)

INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/repos/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl "HTTP/1.1 200 OK"


Initialized MLflow to track repo "michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl"

INFO:dagshub:Initialized MLflow to track repo "michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl"


Repository michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl initialized!

INFO:dagshub:Repository michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl initialized!


In [29]:
mlflow.autolog()

2025/06/27 22:42:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/06/27 22:42:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/06/27 22:42:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


In [30]:
features = list(df.columns)
features.remove("price")

features

['ram_gb',
 'ssd',
 'hdd',
 'graphic_card_gb',
 'warranty',
 'brand_asus',
 'brand_dell',
 'brand_hp',
 'brand_lenovo',
 'brand_other',
 'processor_brand_amd',
 'processor_brand_intel',
 'processor_brand_m1',
 'processor_name_core i3',
 'processor_name_core i5',
 'processor_name_core i7',
 'processor_name_other',
 'processor_name_ryzen 5',
 'processor_name_ryzen 7',
 'ram_type_ddr4',
 'ram_type_other',
 'os_other',
 'os_windows',
 'os_bit_32-bit',
 'os_bit_64-bit',
 'weight_casual',
 'weight_gaming',
 'weight_thinnlight',
 'touchscreen_0',
 'touchscreen_1']

In [31]:
X = df[features]

In [32]:
len(features)

30

In [33]:
y = df["price"]
y

0       2321
1       2613
2       2680
3       4689
4       1808
       ...  
775     9111
776     9714
777    10049
778     9580
779     3852
Name: price, Length: 780, dtype: int64

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [35]:
def evaluate_and_log_model(kind, model_name, model, X_test, y_test):
    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    mape = mean_absolute_percentage_error(y_test, predictions)

    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("R2", r2)
    mlflow.log_metric("MAPE", mape)

    signature = infer_signature(X_test, predictions)

    if kind == "catboost":
        mlflow.catboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "xgboost":
        mlflow.xgboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "lightgbm":
        mlflow.lightgbm.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    else:
        mlflow.sklearn.log_model(model, model_name, signature=signature, input_example=X_test[:5])

    print(f"Model {model_name} logged with MSE: {mse}, MAE: {mae}, R2: {r2}, MAPE: {mape}")

### Experimento com Ridge Regression

In [27]:
with mlflow.start_run(run_name="Ridge Regression"):
    param_grid = {
        'alpha': [0.1, 1.0, 10.0, 100.0],
        'fit_intercept': [True, False],
        }
    
    ridge = Ridge()

    grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False))
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    mlflow.log_param("best_alpha", grid_search.best_params_['alpha'])
    mlflow.log_param("best_fit_intercept", grid_search.best_params_['fit_intercept'])

    evaluate_and_log_model("sklearn", "ridge_regression", best_model, X_test, y_test)

2025/06/17 22:47:02 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/michelpf/Documents/Projects/fiap-ds-mlops-10dtsr-laptop-pricing-brl/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/06/17 22:47:02 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered 

🏃 View run able-yak-400 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/246a068944994f07b3a5d49a0d9c4e3d
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run burly-shoat-601 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/9c7d5a030c9e44fd96f77d9ee48715f2
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run popular-roo-974 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/210e1a8b22bb495fa4d88bb18b4e7978
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run fortunate-kit-33 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/c45df76fc26746039d313e122c8952c7
🧪 View experimen

/Users/michelpf/Documents/Projects/fiap-ds-mlops-10dtsr-laptop-pricing-brl/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Model ridge_regression logged with MSE: 3024089.113899021, MAE: 1137.7871730265838, R2: 0.6421034856259346, MAPE: 0.22065408855367996
🏃 View run Ridge Regression at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/fa22710c2eb9419baf9e48b90c66df33
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0


🏃 View run intelligent-tern-610 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/ce1316e17fdd464d90f2891a936521a7
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run resilient-shrike-134 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/8412d9e91d304667bab3aba1e29bcc8e
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run zealous-lamb-848 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/b369ed2e54134df3971d7f10a15fa82c
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run brawny-bat-778 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/89478e54cea0478dbcd3d8eb27861aa7
🧪 Vi

### Decision Tree Regressor

In [28]:
with mlflow.start_run(run_name="Decision Tree Regression"):
    param_grid = {
        'max_depth': [None, 3, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    ridge = DecisionTreeRegressor()

    grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False))
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    mlflow.log_param("best_max_depth", grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_split", grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_['min_samples_leaf'])

    evaluate_and_log_model("sklearn", "ridge_regression", best_model, X_test, y_test)

2025/06/17 23:04:25 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/michelpf/Documents/Projects/fiap-ds-mlops-10dtsr-laptop-pricing-brl/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/06/17 23:04:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered 

Model ridge_regression logged with MSE: 3668953.253182008, MAE: 1028.2912090410825, R2: 0.5657847598868493, MAPE: 0.16346935432919718
🏃 View run Decision Tree Regression at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/5314415177954dfcb35ae1ff8cf0721d
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0


## XGBoost

In [18]:
with mlflow.start_run(run_name="XGBoost_Regressor_Advanced"):
   
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'gamma': [0, 1],
        'reg_alpha': [0, 0.1],
        'reg_lambda': [1, 5],
        'min_child_weight': [1, 3]
    }
   
    xgb = XGBRegressor(random_state=42, verbosity=0)
    grid_search = GridSearchCV(xgb, param_grid, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False), cv=5)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    mlflow.log_param("best_n_estimators", best_model.n_estimators)
    mlflow.log_param("best_max_depth", best_model.max_depth)
    mlflow.log_param("best_learning_rate", best_model.learning_rate)
    evaluate_and_log_model("xgboost", "XGBoost Regressor", best_model, X_test, y_test)

2025/06/24 21:50:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/michelpf/Documents/Projects/fiap-ds-mlops-10dtsr-laptop-pricing-brl/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/06/24 22:01:40 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered 

🏃 View run incongruous-cub-245 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/59874b38ec9744ca859723a03efd5397
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run rebellious-smelt-121 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/46a6050bb42945c5a7a99f5f3641ed6e
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run upset-pig-375 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/588b9a0b139248c4ba15118bee027df3
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run sassy-squid-688 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/466c9f4614b246ee912b2fc548cda5c4
🧪 View 

/Users/michelpf/Documents/Projects/fiap-ds-mlops-10dtsr-laptop-pricing-brl/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/Users/michelpf/Documents/Projects/fiap-ds-mlops-10dtsr-laptop-pricing-brl/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1028: UserWarning: [22:02:15] WARNING: /Users/run

Model XGBoost Regressor logged with MSE: 2622710.25, MAE: 878.944580078125, R2: 0.6896060705184937, MAPE: 0.13514773547649384
🏃 View run XGBoost_Regressor_Advanced at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/9fcf1bcf27d946c98f857a2a9af74e26
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0


## Nova abordagem XGBoost

In [36]:
with mlflow.start_run(run_name="XGBoost_Regressor"):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.1, 0.2, 0.3]
    }
    xgb = XGBRegressor(random_state=42, verbosity=0)
    grid_search = GridSearchCV(xgb, param_grid, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False), cv=5)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    mlflow.log_param("best_n_estimators", best_model.n_estimators)
    mlflow.log_param("best_max_depth", best_model.max_depth)
    mlflow.log_param("best_learning_rate", best_model.learning_rate)
    evaluate_and_log_model("xgboost", "XGBoost Regressor", best_model, X_test, y_test)

2025/06/27 22:43:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/michelpf/Documents/Projects/fiap-ds-mlops-10dtsr-laptop-pricing-brl/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/06/27 22:44:59 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered 

Model XGBoost Regressor logged with MSE: 2609672.0, MAE: 889.0443115234375, R2: 0.6911491751670837, MAPE: 0.14123447239398956
🏃 View run XGBoost_Regressor at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/2e5d588a429649d1923868dddf7c586d
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0


## Registro de Modelo em Produção

In [ ]:
run_id = "9fcf1bcf27d946c98f857a2a9af74e26"

mlflow.register_model(model_uri=f"runs:/{run_id}/model", name="laptop-pricing-model-brl")

Registered model 'laptop-pricing-model-brl' already exists. Creating a new version of this model...
2025/06/24 22:12:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: laptop-pricing-model-brl, version 3
Created version '3' of model 'laptop-pricing-model-brl'.


<ModelVersion: aliases=[], creation_timestamp=1750813940565, current_stage='None', description='', last_updated_timestamp=1750813940565, name='laptop-pricing-model-brl', run_id='9fcf1bcf27d946c98f857a2a9af74e26', run_link='', source='mlflow-artifacts:/d4bd9539a7fb4a12aa16e3387cfaf1ac/9fcf1bcf27d946c98f857a2a9af74e26/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='3'>